#### **Load table from silver schema to gold schema**

In [1]:
spark.read.table("silver.churn").write.format("delta").saveAsTable("gold.churn")
spark.read.table("silver.customers").write.format("delta").saveAsTable("gold.customers")
spark.read.table("silver.orders").write.format("delta").saveAsTable("gold.orders")
spark.read.table("silver.products").write.format("delta").saveAsTable("gold.products")
spark.read.table("silver.sales").write.format("delta").saveAsTable("gold.sales")

StatementMeta(, 95239827-78b7-4f14-9487-5e38cc657551, 3, Finished, Available, Finished)

#### **Create Calender table**

In [6]:
from pyspark.sql.functions import (
    explode, sequence, to_date, expr,
    year, quarter, month, date_format,
    weekofyear, dayofmonth
)

StatementMeta(, 95239827-78b7-4f14-9487-5e38cc657551, 8, Finished, Available, Finished)

In [7]:
bound = spark.read.table("silver.orders").select(min("OrderDate").alias("min_date")
                                                , max("OrderDate").alias("max_date")).collect()[0]

StatementMeta(, 95239827-78b7-4f14-9487-5e38cc657551, 9, Finished, Available, Finished)

In [8]:
bound

StatementMeta(, 95239827-78b7-4f14-9487-5e38cc657551, 10, Finished, Available, Finished)

Row(min_date=datetime.date(2022, 6, 1), max_date=datetime.date(2022, 12, 1))

In [9]:
calendar = (
    spark.createDataFrame(
        [(bound[0], bound[1])],
        ["start_date", "end_date"]
    )
    .select(
        explode(
            sequence(
                to_date("start_date"),
                to_date("end_date"),
                expr("INTERVAL 1 DAY")
            )
        ).alias("date")
    )
    .withColumn("year", year("date"))
    .withColumn("quarter", quarter("date"))
    .withColumn("month", month("date"))
    .withColumn("month_name", date_format("date", "MMMM"))
    .withColumn("week", weekofyear("date"))
    .withColumn("day", dayofmonth("date"))
    .withColumn("day_name", date_format("date", "EEEE"))
)

display(calendar.limit(5))

StatementMeta(, 95239827-78b7-4f14-9487-5e38cc657551, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9cdd1919-1807-4d6f-8700-395f0e070269)

In [10]:
calendar.write.format("delta").saveAsTable("gold.calendar")

StatementMeta(, 95239827-78b7-4f14-9487-5e38cc657551, 12, Finished, Available, Finished)